In [10]:
# Import the required libraries and dependencies
import os
import pandas as pd
import plotly.express as px
import hvplot.pandas
from pathlib import Path
from dotenv import load_dotenv

## Enable your Mapbox API access token

In [11]:
# Load the .env file into the notebook
load_dotenv()

# Read in your MAPBOX_API_KEY
mapbox_api_access_token = os.getenv('MAPBOX_API_ACCESS_TOKEN')

# Confirm the availability of your Mapbox API access token by checking its type
display(type(mapbox_api_access_token))

# Set your Mapbox API access token
px.set_mapbox_access_token(mapbox_api_access_token)

str

## Import the data 

In [34]:
# Import large kickstarter recent dataset
kickstarter_large_recent_df = pd.read_csv(
    Path('./Resources/kickstarter_large_dataset/ks-projects-201801.csv')
)

# Imported data has a space at the end of the column name
# Remove spaces in columns name
kickstarter_large_recent_df.columns = kickstarter_large_recent_df.columns.str.replace(' ','')

# Set the index as the ID
kickstarter_large_recent_df.set_index('ID', inplace=True)

# View head
kickstarter_large_recent_df.head()

In [57]:
# Import small kickstarter live dataset
kickstarter_small_live_df = pd.read_csv(
    Path('./Resources/kickstarter_small_dataset/live.csv')
)

# Fix ID column
kickstarter_small_live_df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

# Set the index as the ID
kickstarter_small_live_df.set_index('ID', inplace=True)

# View head
kickstarter_small_live_df.head()

In [60]:
# Import small kickstarter most backed dataset
kickstarter_small_most_backed_df = pd.read_csv(
    Path('./Resources/kickstarter_small_dataset/most_backed.csv')
)

# Fix ID column
kickstarter_small_most_backed_df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

# Set the index as the ID
kickstarter_small_most_backed_df.set_index('ID', inplace=True)

# View head
kickstarter_small_most_backed_df.head()

In [64]:
# Import indiegogo dataset
indiegogo_df = pd.read_csv(
    Path('./Resources/indiegogo.csv')
)

# Fix ID column
indiegogo_df.rename(columns={'project_id' : 'ID'}, inplace=True)

# Set the index as the ID
indiegogo_df.set_index('ID', inplace=True)

# View head
indiegogo_df.head()